# The add_interface method <3

Using data from the Buzsaki data share: https://buzsakilab.nyumc.org/datasets/WatsonBO/BWRat17/BWRat17_121712/

In [1]:
from pathlib import Path
from pprint import pprint

from nwb_conversion_tools import NWBConverter, spec
from nwb_conversion_tools.interfaces import list_interfaces
import pynwb


The base path is the root of a single data directory -- later we will apply it to any number of directories :)

In [2]:
base_path = Path('~/Desktop/watson/BWRat17/BWRat17_121712').expanduser()

In [3]:
converter = NWBConverter(base_path)

# Specify Metadata
We specify the general structure and location of the data and metadata within a folder, rather than loading it explicitly


Let's tell it where to find that medaaaadadaaaa. Our session id is the full name of a file named, in this case `'BWRat17_121712.lfp'`, and we want to store this as some metadata nested within `'NWBFile'` -- so let's tell the converter that!

We use the `spec.Path` specifier to extract metadata from a path

In [4]:
sid_spec = spec.Path('{NWBFile[identifier]}.lfp')
sid_spec.parse(base_path)

{'NWBFile': {'identifier': 'BWRat17_121712'}}

Fabulous! now we can add that to our converter!!!

In [5]:
converter.add_metadata(sid_spec)

Keep doin that huh? add some more metadata! This dataset uses the date as the `session_start_time` parameter. As a bonus, we can specify multiple pieces of metadata in a single spec -- they get merged at the end don't worry!

In [6]:
start_spec = spec.Path('{Subject[subject_id]}_{NWBFile[session_start_time]}.lfp')
pprint(start_spec.parse(base_path))
converter.add_metadata(start_spec)

{'NWBFile': {'session_start_time': '121712'},
 'Subject': {'subject_id': 'BWRat17'}}


Static metadata is also fine

In [7]:
converter.add_metadata({'NWBFile':{'institution':'NYU'}})

# Specifying Interfaces


To see the available interfaces, we can just ask!

In [8]:
# list all interfaces
pprint(list_interfaces()[0:10])

[<class 'nwb_conversion_tools.interfaces.imaging.imaging.TiffImagingInterface'>,
 <class 'nwb_conversion_tools.interfaces.imaging.imaging.Hdf5ImagingInterface'>,
 <class 'nwb_conversion_tools.interfaces.imaging.imaging.SbxImagingInterface'>,
 <class 'nwb_conversion_tools.interfaces.recording.blackrock.BlackrockRecordingExtractorInterface'>,
 <class 'nwb_conversion_tools.interfaces.recording.ced.CEDRecordingInterface'>,
 <class 'nwb_conversion_tools.interfaces.recording.intan.IntanRecordingInterface'>,
 <class 'nwb_conversion_tools.interfaces.recording.neuroscope.NeuroscopeRecordingInterface'>,
 <class 'nwb_conversion_tools.interfaces.recording.neuroscope.NeuroscopeMultiRecordingTimeInterface'>,
 <class 'nwb_conversion_tools.interfaces.recording.open_ephys.OpenEphysRecordingExtractorInterface'>,
 <class 'nwb_conversion_tools.interfaces.recording.spike_glx.SpikeGLXRecordingInterface'>]


In [9]:
# list interfaces of a specific category
pprint(list_interfaces('imaging'))

[<class 'nwb_conversion_tools.interfaces.imaging.imaging.TiffImagingInterface'>,
 <class 'nwb_conversion_tools.interfaces.imaging.imaging.Hdf5ImagingInterface'>,
 <class 'nwb_conversion_tools.interfaces.imaging.imaging.SbxImagingInterface'>]


In [10]:
# and get a specific interface
tiff_interface = list_interfaces('imaging', 'tiff')
tiff_interface

nwb_conversion_tools.interfaces.imaging.imaging.TiffImagingInterface

In [11]:
# the search depends on the class attributes
print('\n'.join((
    tiff_interface.interface_type, 
    tiff_interface.device_name))
     )

imaging
tiff


## Adding a Neuroscope Sorting Interface

Using that syntax, we can programmatically add an interface to our dataset spec. First we can query what parameters we need to add it

In [12]:
converter.add_interface('sorting', 'neuroscope')

Source Schema for ABCMeta
-------------------------
{'additionalProperties': False,
 'properties': {'exclude_shanks': {'type': 'array'},
                'folder_path': {'type': 'string'},
                'gain': {'type': 'number'},
                'keep_mua_units': {'default': True, 'type': 'boolean'},
                'load_waveforms': {'default': False, 'type': 'boolean'}},
 'required': ['folder_path'],
 'type': 'object'}
-------------------------


We can specify `folder_path`, `gain`, and some others, but `folder_path` is required..

Let's use another spec -- this time the `Glob` spec, which is sort of the opposite of the `Path` spec -- using a pattern and/or some metadata, specify a file. in this case the data is originally in the base directory, but that's no fun! let's put them in a subdirectory that uses the subject id just to make it a lil more fun.

In [13]:
sub_path = base_path / "BWRat17_121712"
sub_path.mkdir(exist_ok=True)

matches = list(base_path.glob('*.clu*'))
matches.extend(list(base_path.glob('*.res*')))
matches.extend(list(base_path.glob('*.xml')))
matches.extend(list(base_path.glob('*.spk*')))

for match in matches:
    match.rename(sub_path / match.name)

Now we use the `Glob`!

In [14]:
converter.add_interface(
    'sorting', 'neuroscope', 
    spec.Glob('folder_path', 
              '{NWBFile[identifier]}'))

In [15]:
output_path = Path('~').expanduser() / 'test_nwb.nwb'
converter.run_conversion(nwbfile_path=str(output_path))

/Users/jonny/git/nwb-conversion-tools/venv/lib/python3.8/site-packages/pynwb/file.py:753: UserWarning: Date is missing timezone information. Updating to local timezone.
  warn("Date is missing timezone information. Updating to local timezone.")


NWB file saved at /Users/jonny/test_nwb.nwb!


root pynwb.file.NWBFile at 0x6041633504
Fields:
  file_create_date: [datetime.datetime(2021, 3, 31, 23, 8, 5, 401353, tzinfo=tzlocal())]
  identifier: 54a51c63-1eb3-4fd8-ae7e-6a6d29d3b5e7
  institution: NYU
  session_description: no description
  session_start_time: 1970-01-01 00:00:00-08:00
  subject: subject pynwb.file.Subject at 0x6041633264
Fields:
  subject_id: BWRat17

  timestamps_reference_time: 1970-01-01 00:00:00-08:00
  units: units <class 'pynwb.misc.Units'>

Did it work? Our converter seems to think so

In [16]:
converter.get_metadata()

{'NWBFile': {'session_description': 'no description',
  'session_start_time': datetime.datetime(1970, 1, 1, 0, 0),
  'identifier': '0a2fe04b-0b75-40fe-9ff5-d7dc67651e5a'},
 'Ecephys': {'Device': [{'description': 'BWRat17_121712.xml'}],
  'ElectrodeGroup': [{'name': 'shank1', 'description': 'shank1 electrodes'},
   {'name': 'shank2', 'description': 'shank2 electrodes'},
   {'name': 'shank3', 'description': 'shank3 electrodes'},
   {'name': 'shank4', 'description': 'shank4 electrodes'},
   {'name': 'shank5', 'description': 'shank5 electrodes'},
   {'name': 'shank6', 'description': 'shank6 electrodes'},
   {'name': 'shank7', 'description': 'shank7 electrodes'},
   {'name': 'shank8', 'description': 'shank8 electrodes'},
   {'name': 'shank9', 'description': 'shank9 electrodes'}],
  'Electrodes': [{'name': 'shank_electrode_number',
    'description': '0-indexed channel within a shank.',
    'data': [0,
     1,
     2,
     3,
     4,
     5,
     6,
     7,
     0,
     1,
     2,
     3,
  

Woohoo! we got 'em!

In [17]:
io = pynwb.NWBHDF5IO(str(output_path), 'r')
nwbfile_in = io.read()
print(nwbfile_in.units)

units pynwb.misc.Units at 0x6069716880
Fields:
  colnames: ['spike_times']
  columns: (
    spike_times_index <class 'hdmf.common.table.VectorIndex'>,
    spike_times <class 'hdmf.common.table.VectorData'>
  )
  description: Autogenerated by NWBFile
  id: id <class 'hdmf.common.table.ElementIdentifiers'>
  waveform_unit: volts



# TODO

* Now that we've got an abstract representation of a dataset, we should be able to save it, load it, etc.
* It is also trivial to then apply it to a list of directories by making a simple `apply`-like method.  
* ??? lots more development ???

like this is what i'm on about:
```
converter = NWBConverter.from_spec('spec_file.pck')
converter.apply('/data/experiments/*')
```